In [2]:
# Import Necessary Libraries

from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os

# Load .env file

load_dotenv()

# Set the TESSDATA_PREFIX environment variable

tessdata = os.getenv("TESSDATA_PREFIX")
os.environ['TESSDATA_PREFIX'] = tessdata

# Set the Google api key

my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [18]:

%%time
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,GoogleGenerativeAI

model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0,max_output_tokens=200)


CPU times: total: 0 ns
Wall time: 3 ms


In [4]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python", "js", "golang"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = model.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
chain = prompt | structured_llm


In [29]:
from langchain_core.prompts import PromptTemplate

python_chain = PromptTemplate.from_template(
    """You are an expert in python. \
Respond to the following question:

Question: {question}
Answer:"""
) | model
js_chain = PromptTemplate.from_template(
    """You are an expert in js. \
Respond to the following question:

Question: {question}
Answer:"""
) | model
golang_chain = PromptTemplate.from_template(
    """You are an expert in golang. \
Respond to the following question:
Question: {question}
Answer:"""
) | model

In [30]:
def choose_route(question):
    result = chain.invoke({"question": question})
    if "python" in result.datasource.lower():
        ### Logic here 
        print("Using Python")
        return python_chain
    elif "js" in result.datasource.lower():
        ### Logic here 
        print("Using js")
        return js_chain
    else:
        ### Logic here 
        print("Using Golang")
        return golang_chain

from langchain_core.runnables import RunnableLambda

#full_chain = router | RunnableLambda(choose_route)

In [31]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    choose_route ) |StrOutputParser()

In [32]:

%%time
result = full_chain.invoke({"question": "How to work with Python in windows"})


Using Python
CPU times: total: 0 ns
Wall time: 3.13 s


In [33]:
%%time
result1 = model.invoke("How to work with Python in windows")
print(result1.usage_metadata)

{'input_tokens': 8, 'output_tokens': 200, 'total_tokens': 208}
CPU times: total: 0 ns
Wall time: 1.9 s
